In [59]:
import pandas as pd
import numpy as np
import math
import time
import csv
import re
import json
import codecs
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions 
from pyspark.sql import Row
from collections import OrderedDict
from bs4 import BeautifulSoup,SoupStrainer




In [60]:
# parsing data from sgm file

from bs4 import BeautifulSoup,SoupStrainer


def readData(fileNmae):
    #print("parsing start")
    cal = 0
    data = ''
    # avoid unicode error 
    with codecs.open(fileName, "r", encoding='utf-8', errors='ignore') as f:
        soup = BeautifulSoup(f, 'html.parser')
        contents = soup.find_all('body')
        for content in contents:
            data = data + content.get_text()
            cal +=1
        #print("parsing",fileName,"done!")
        return data
        
    #print(cal)
    
# parsing data from sgm file

In [61]:
# Q1

sTime = time.time()

file = ["000","001","002","003","004","005","006","007","008","009","010","011","012","013","014","015","016","017","018","019","020","021"]

for i in range(1):
    fileName = "/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-"
    fileName = fileName + file[i] + ".sgm"
    data = readData(fileName)
    ans = []
    print("k-shingle start")
    # retrieval all words on <body> tag
    for content in data.split("\n"):
        content = content.strip()
        # save each shingles 
        shinglesByWord = set()
        # split each body content by space
        wordList = content.split(" ")
        # retrieval each body content
        for w in range(0,len(wordList)-1):
            newShingles = wordList[w] + ',' + wordList[w+1]
            # add 2-shingle to set()
            if newShingles not in shinglesByWord:
                shinglesByWord.add(newShingles)
        for s in shinglesByWord:
            ans.append(s)
            
        #print(shinglesByWord)
    
    #write result to txt
    Q1Ans = "Q1.txt"
    with open(Q1Ans, 'a') as out:
        out.write(json.dumps(result))

        
        
        #print(shinglesByWord)
    print("k-shingle done!\n")
    
print('Processing Q1 took %.2f sec.' % (time.time() - sTime))

# Q1


k-shingle start
{'throughout,the', 'continued,throughout', 'Showers,continued', 'the,week', 'week,in'}
{'alleviating,the', 'cocoa,zone,', 'Bahia,cocoa', 'since,early', 'the,Bahia', 'drought,since', 'the,drought', 'zone,,alleviating'}
{'prospects,for', 'January,and', 'the,coming', 'coming,temporao,', 'improving,prospects', 'for,the', 'and,improving'}
{'been,restored,', 'humidity,levels', 'normal,humidity', 'levels,have', 'not,been', 'although,normal', 'have,not'}
{'weekly,review.', 'Comissaria,Smith', 'its,weekly', 'Smith,said', 'said,in', 'in,its'}
{'means,the', 'temporao,will', 'will,be', 'this,year.', 'the,temporao', 'dry,period', 'late,this', 'period,means', 'be,late', 'The,dry'}
{'ended,February', 'February,22', 'were,155,221', '22,were', 'the,week', 'for,the', '155,221,bags', 'Arrivals,for', 'week,ended'}
{'making,a', 'season,of', 'of,60', 'kilos,making', 'a,cumulative', 'total,for', 'for,the', '60,kilos', 'of,5.93', 'cumulative,total', 'the,season'}
{'stage,last', 'at,the', 'Agai

{'dlr,jump', 'ended,February', 'jump,in', 'in,the', 'February,16.', 'the,week', 'week,ended'}
{'the,Fed', 'have,to', 'Looking,ahead,', 'Fed,will', 'to,tread', 'will,have', 'economists,said', 'tread,a', 'said,the', 'ahead,,economists'}
{'between,the', 'progress,in', 'fine,line', 'in,the', "dollar's,progress", 'the,international', 'line,between', "the,dollar's"}
{'the,development', 'currency,markets', 'markets,and', 'of,the', 'the,domestic', 'and,the', 'development,of', 'domestic,economy.'}
{'has,perhaps', '"The,market', 'market,has', 'exaggerated,the', 'effect,on', "dollar's,effect", 'perhaps,exaggerated', "the,dollar's"}
{'Johnson.,"Of', '"Of,course,', "First,Chicago's", 'said,First', 'course,,it', 'policy,",said', "Chicago's,Johnson.", 'it,will', 'Fed,policy,"'}
{'in,future', 'future,policy', 'decisions,but', 'take,the', 'dollar,into', 'but,if', 'the,dollar', 'into,account', 'account,in', 'policy,decisions'}
{"it,won't", 'the,economy', 'is,weak,', 'economy,is', 'back,from', "won't,pul

{'bank,said.'}
{'or,less', 'maturities,of', 'ceiling,on', 'MMCs,with', 'on,MMCs', 'But,the', 'year,or', 'with,maturities', 'the,ceiling', 'of,one', 'one,year'}
{'0.75,percentage', 'the,CD', 'below,the', 'remain,the', 'will,remain', 'percentage,point', 'same,,or', 'or,0.75', 'the,same,', 'point,below'}
set()
{'The,minimum', 'will,be', 'be,lowered', 'to,10', 'minimum,denomination', 'MMCs,will', '10,mln', 'lowered,to', 'of,MMCs', 'denomination,of'}
{'30,mln.', 'from,30', 'yen,from'}
{'The,bank', 'bank,also', 'also,said', 'said,time', 'will,be', 'be,lowered', 'lowered,by', 'rates,will', 'deposit,rates', 'time,deposit'}
{'effective,March', '16,,in', 'March,16,', 'half-point,cut', 'with,the', 'point,,effective', '0.37,point,', 'the,half-point', 'line,with', 'in,line'}
{'in,the', 'official,discount', 'the,official', 'on,February', 'rate,on', 'discount,rate', 'February,23.'}
{'deposits,at', 'rates,on', 'and,one-year', 'on,two-year', 'one-year,deposits', 'two-year,and', 'interest,rates', 'New,i

{'maize,to', 'non-EC,countries.', 'tonnes,of', 'of,free', 'sale,of', 'market,maize', 'of,15,000', 'free,market', '15,000,tonnes', 'to,non-EC'}
{'for,barley', 'Requests,for', 'barley,were', 'for,export', 'were,for', 'Switzerland,,Austria', 'export,to', 'to,Switzerland,'}
{'and,Melilla', 'and,Lichtenstein,', 'an,unchanged', 'Lichtenstein,,Ceuta', 'unchanged,pre-fixed', 'Melilla,at', 'Ceuta,and', 'at,an'}
{'restitution,of', 'while,requests', 'of,125', 'for,soft', '125,Ecus', 'requests,for', 'a,tonne,', 'Ecus,a', 'tonne,,while', 'soft,wheat'}
{'flour,were', 'non-EC,countries', 'to,various', 'countries,at', 'for,export', 'various,non-EC', 'were,for', 'export,to', 'at,an'}
{'a,tonne.', 'Ecus,a', 'unchanged,178', '178,Ecus'}
set()
{'\x03A,study', 'group,said', 'study,group', 'said,the', 'United,States', 'the,United'}
{'to,one', 'increase,its', 'reserve,to', 'one,mln', 'should,increase', 'its,strategic', 'petroleum,reserve', 'strategic,petroleum'}
{'present,and', 'barrels,as', 'as,one', 'the,p

{'Co,Ltd', 'with,Matsushita', 'Ltd,of', 'Matsushita,Electric', 'Electric,Industrial', 'Industrial,Co', 'of,Japan.', 'automation,with'}
{'The,company', 'to,combine', "combine,Matsushita's", 'company,said', 'the,venture', 'venture,aims', 'said,the', 'aims,to'}
{'manufacturing,and', 'experience,in', 'electronics,manufacturing', 'high-volume,electronics', 'in,high-volume'}
{'integrated,manufacturing.', 'of,computer', 'computer,integrated', "Westinghouse's,knowledge", 'knowledge,of'}
{'venture,will', 'design,,sell', 'service,automated', 'will,design,', 'and,service', 'The,venture', 'sell,and'}
{'operations,by', 'is,expected', 'expected,to', 'systems,,and', 'and,is', 'manufacturing,systems,', 'to,begin', 'begin,operations'}
{'summer,,it', 'the,summer,', 'it,added.'}
set()
{'\x03Efforts,to', 'to,break', 'break,an', 'impasse,between', 'an,impasse'}
{'and,importing', 'exporting,and', 'importing,countries', 'coffee,exporting', 'over,regulating', 'countries,over', 'regulating,the'}
{'the,face', '

{'next,week,', 'draining,8.5', 'An,earlier', 'earlier,pact', '8.5,billion', 'pact,expires', 'expires,next', 'week,,draining'}
{'from,the', 'system.,Banks', 'heavy,but', 'Banks,also', 'face,a', 'a,heavy', 'but,temporary', 'the,system.', 'also,face', 'marks,from'}
{'a,major', 'for,customers.', 'month,from', 'drain,this', 'tax,deadline', 'this,month', 'from,a', 'major,tax', 'deadline,for'}
{'at,the', '52.0,billion', 'billion,marks', 'held,52.0', 'on,February', 'Banks,held', '27,at', 'February,27', 'marks,on'}
{'month,,just', 'over,the', 'averaging,51.0', '51.0,billion', 'billion,over', 'the,whole', 'Bundesbank,,averaging', 'whole,month,'}
{'clear,of', 'of,the', 'the,50.5', 'reserve,requirement.', 'February,reserve', 'billion,February', '50.5,billion'}
{'traded,today', 'today,at', '3.85/95,pct,', 'pct,,up', 'from,3.80/90', 'at,3.85/95', 'Call,money', 'money,traded', 'up,from'}
set()
set()
{'Manhattan,Asia', 'Asia,Ltd', 'Ltd,of', 'Kong,has', '\x03Chase,Manhattan', 'Hong,Kong', 'of,Hong'}
{'